In [1]:
import numpy as np
import pandas as pd
import pyodbc

In [2]:
sections = ["Food","NonCatDrinks","Cigar"]
dic_sec = {
    "Food": range(11111,12219),
    "NonCatDrinks" : [21111],
    "Cigar": range(22111,23112)
}
col_name = {
    'Address': 'HHID',
    'DYCOL01': 'Code',
    'DYCOL02': 'BuyingMethod',
    'DYCOL03': 'Grams',
    'DYCOL04': 'Kilos',
    'DYCOL05': 'price_per',
    'DYCOL06': 'Price'
}
years = list(range(89, 99))+ [99] + list(range(1400, 1403))
years = [1403]
for year in years:
    # Path to your .accdb file
    file_path = fr'C:\Users\Mahsa\Desktop\geo\raw_data\{year}\Outfile-{year}.accdb'
    # Connection string (for 64-bit Access Database Engine)
    conn_str = (
        r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
        f'DBQ={file_path};'
    )
    # Connect
    conn = pyodbc.connect(conn_str)
    # List all table names
    tables = [table.table_name for table in conn.cursor().tables(tableType='TABLE')]
    print(f"Year is :{str(year)}".center(50))
    #---------------------------------------------------------------------------------
    
    for section in sections:
        if section == "Food":
            Rdata = pd.read_sql(f"SELECT Address,DYCOL01,DYCOL06 FROM R{year}P3S01", conn)
            Udata = pd.read_sql(f"select Address,DYCOL01,DYCOL06 from U{year}P3S01", conn)
        else:
            Rdata = pd.read_sql(f"SELECT Address,DYCOL01,DYCOL06 FROM R{year}P3S02", conn)
            Udata = pd.read_sql(f"select Address,DYCOL01,DYCOL06 from U{year}P3S02", conn)
        
        data = pd.concat([Rdata, Udata], axis=0)
        data.rename(columns= col_name, inplace=True)
        
        data["HHID"] = data["HHID"].astype("str")
        mask = data["HHID"].str.contains(r"\.")
        data.loc[mask, 'HHID'] = data.loc[mask, "HHID"].str.split(".").str[0]
        data = data.loc[data["HHID"].notna()]
        
        data["Code"] = data["Code"].astype("str")
        data["Code"] = data["Code"].str.lstrip("0")
        data["Code"] = pd.to_numeric(data["Code"], errors='coerce')
        data = data.loc[data["Code"].isin(dic_sec[section])]
        
        data["Price"] = data["Price"].astype("str")
        data["Price"] = data["Price"].str.lstrip("0")
        data["Price"] = pd.to_numeric(data["Price"], errors='coerce')
        data = data.loc[data["Price"].notna()]
        
        data = data[["HHID","Price"]]
        
        agg_dict = {
            'HHID': ('HHID', 'first'),
            f"{section}Exp": ("Price", "sum")
        }

        df = data.groupby("HHID").aggregate(**agg_dict).reset_index(drop=True)
        df = df.fillna(0)
                
        df.to_pickle(fr'{section}_{year}')

                  Year is :1403                   


C:\Users\Mahsa\AppData\Local\Temp\ipykernel_16936\3000899581.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Rdata = pd.read_sql(f"SELECT Address,DYCOL01,DYCOL06 FROM R{year}P3S01", conn)
C:\Users\Mahsa\AppData\Local\Temp\ipykernel_16936\3000899581.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Udata = pd.read_sql(f"select Address,DYCOL01,DYCOL06 from U{year}P3S01", conn)
C:\Users\Mahsa\AppData\Local\Temp\ipykernel_16936\3000899581.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Rdata = pd.read_sql(f"SELECT 